In [32]:
import pandas as pd
import altair as alt

In [33]:


pew= pd.read_excel('Pew Issue Polling 2022.xlsx')

PermissionError: [Errno 13] Permission denied: 'Pew Issue Polling 2022.xlsx'

In [ ]:
pew

In [56]:
republican = pew[['issue','Repub','Repub.1','Repub.2']].copy()
republican['party'] = 'Republicans'
republican = republican.rename(columns={"Repub": "March", "Repub.1": "August", "Repub.2": "October"})

In [57]:
democrat = pew[['issue','Dem','Dem.1','Dem.2']].copy()
democrat['party'] = 'Democrats'
democrat = democrat.rename(columns={"Dem": "March", "Dem.1": "August", "Dem.2": "October"})

In [58]:
final = pd.concat([republican, democrat]).reset_index(drop=True).sort_values(by='issue')
final

,issue,March,August,October,party
0,Abortion,0.40,0.41,0.39,Republicans
18,Abortion,0.46,0.71,0.75,Democrats
11,Climate change,0.14,0.13,0.09,Republicans
29,Climate change,0.64,0.66,0.68,Democrats
7,Economy,0.90,0.90,0.92,Republicans
25,Economy,0.68,0.67,0.65,Democrats
1,Education,0.53,0.52,0.60,Republicans
19,Education,0.66,0.64,0.66,Democrats
6,Energy policy,0.62,0.51,0.63,Republicans
24,Energy policy,0.58,0.56,0.63,Democrats


In [59]:
# use pd.melt to get the data into "long form" for they dynamic altair viz
changedata = pd.melt(final, id_vars=['party','issue'], value_vars=['March','August','October'])
changedata

,party,issue,variable,value
0,Republicans,Abortion,March,0.40
1,Democrats,Abortion,March,0.46
2,Republicans,Climate change,March,0.14
3,Democrats,Climate change,March,0.64
4,Republicans,Economy,March,0.90
...,...,...,...,...
103,Democrats,The coronavirus outbreak,October,0.32
104,Democrats,Violent crime,October,0.45
105,Republicans,Violent crime,October,0.74
106,Republicans,Voting policies,October,0.57


In [66]:
#Generate dynamic Altair chart


def generateLineChartP21(changedata):

    #dropdown options for sorting data by indicator
    metricOptions = ['Republicans',
       'Democrats']
    
    input_dropdown = None
    
    dropdown_selection = None
    
    #input the dropdown menu and bind it to the dropdown selection that uses the indicators as its field
    input_dropdown = alt.binding_select(options=metricOptions, name="party")
    dropdown_selection = alt.selection_single(fields=['party'], init={"party":metricOptions[0]}, bind=input_dropdown)
    

    #create a base line chart with a tooltip that will show the country name, year, and amount for every point on the graph
    line = alt.Chart(changedata).mark_line().encode( 
         x = alt.X('variable:N', title = 'Month of 2022'),
         y = alt.Y('value:Q', title = 'Percentage who consider issue very important for their vote'),
         tooltip=['issue:N','variable:N','value:Q'],
     
         color = alt.Color('issue', 
                  )
    )
     
    #make a selection that will highlight individual lines based on the country name for the point that the mouse is nearest to   
    highlight = alt.selection(type='single', on='mouseover',
                          fields=['issue'], nearest=True)
    
    #mark the invisible points adding the highlight selection that determines which field to highlight based on mouseover proximity
    points1 = line.mark_circle().encode(
    opacity=alt.value(0)
    ).add_selection(
    highlight
    )
      
    #line chart that increases the size of the lines of the base line chart based on the condition of whether the mouse is closest
    #to a point on the highlight selection being used on the points
    lines = line.mark_line().encode(
    size=alt.condition(highlight, alt.value(3), alt.value(1))
    )
    
    #layer the points and lines charts and add the dropdown selection that filters the data accordingly
    final = alt.layer(
        points1, lines).add_selection(
    dropdown_selection
    ).transform_filter(
    dropdown_selection
    ).properties(title = 'Importance of Issues for Voters - PEW', width = 400, height = 500)
    
    return final


In [67]:
generateLineChartP21(changedata)

alt.LayerChart(...)